# **Reto 1**

Departamento de Ingeniería Biomédica

Universidad de los Andes

**IBIO-2440:** Fundamentos del machine learning


**Nombres de los integrantes**


1.  Laura Julieth Carretero Serrano
2.  Juan David Rios Nisperuza

**Número del grupo**

*3*




### **1. Explicación del problema general con palabras.**

El problema trata sobre organizar los turnos de las enfermeras en la Clínica del Río para que ellas tengan un mejor horario y, al mismo tiempo, la clínica gaste lo menos posible en salarios.  

Las enfermeras quieren trabajar cinco días seguidos y descansar dos días seguidos. La clínica ya sabe cuántas enfermeras necesita cada día y cuánto cuesta pagarles. El reto es encontrar la mejor forma de asignar los turnos para que se cumplan estas dos condiciones:  
1. **Cubrir la demanda diaria de enfermeras.**  
2. **Minimizar el costo total de los salarios.**  


### **2. Definición de todos los componentes de un problema de optimización, para los cuales se deben justificar sus planteamientos y sus unidades.**

El modelo de optimización busca asignar enfermeras a cada turno semanal de manera que el costo total de la nómina sea el menor posible, asegurando al mismo tiempo que cada día se cubra con la cantidad mínima requerida de personal.  

### **Función objetivo**  
El costo total de la nómina se obtiene sumando el número de enfermeras asignadas a cada turno, multiplicado por el salario diario correspondiente. En este caso, los salarios varían según el turno, con valores de 1280, 1360 y 1200 unidades monetarias, los cuales son calculados por el salario base para cada enfermera por dia multiplicado por el numero de enfermera necesaria cada dia. La función de costo se expresa como:  

Costo total = 1280 × número de enfermeras en el turno 0 + 1360 × número de enfermeras en el turno 1 + … + 1200 × número de enfermeras en el turno 6.  

El objetivo es encontrar la cantidad de enfermeras en cada turno de manera que este costo sea el menor posible.  

### **Restricciones**  
Cada día de la semana tiene una demanda específica de enfermeras. Para garantizar que esta necesidad se cumpla, se establecen ecuaciones que suman las enfermeras asignadas a los turnos que cubren cada día y las igualan a la cantidad requerida.  

Por ejemplo, el lunes requiere 5 enfermeras. Los turnos que trabajan ese día son los asociados a las variables de decisión de los turnos C, D, E, F, G, (esto debido a que el turno A y B descansan ese dia) por lo que la ecuación de restricción es:  

Número de enfermeras en el turno C + número de enfermeras en el turno D + número de enfermeras en el turno E + número de enfermeras en el turno F + número de enfermeras en el turno G = 5.  

De manera similar, se plantean ecuaciones para el resto de los días, utilizando las combinaciones de turnos que cubren cada jornada.  


### **3. Descripción de la implementación del código utilizado.**

In [397]:
from scipy.optimize import minimize
import numpy as np
import sympy as sym


In [398]:
# Función de costo: minimiza el costo total de la nómina de enfermeras
def costo_nomina(x):
    return 1280*x[0] + 1360*x[1] + 1360*x[2] + 1360*x[3] + 1360*x[4] + 1280*x[5] + 1200*x[6]

# Restricciones de dotación mínima de enfermeras por día
def restriccion_dia_lunes(x): 
    return x[1] + x[2] + x[3] + x[4] + x[5] - 6  # Lunes

def restriccion_dia_martes(x): 
    return x[2] + x[3] + x[4] + x[5] + x[6] - 5  # Martes

def restriccion_dia_miercoles(x): 
    return x[0] + x[3] + x[4] + x[5] + x[6] - 3  # Miércoles

def restriccion_dia_jueves(x): 
    return x[0] + x[1] + x[4] + x[5] + x[6] - 2  # Jueves

def restriccion_dia_viernes(x): 
    return x[0] + x[1] + x[2] + x[5] + x[6] - 3  # Viernes

def restriccion_dia_sabado(x): 
    return x[0] + x[1] + x[2] + x[3] + x[6] - 5  # Sábado

def restriccion_dia_domingo(x): 
    return x[0] + x[1] + x[2] + x[3] + x[4] - 6  # Domingo

# Definir límites de las variables (número de enfermeras por tipo de turno, no negativo)
bounds = [(0, None)] * 7

# Valores iniciales 
x0 = np.ones(7) * 3

restricciones = [
    {'type': 'eq', 'fun': restriccion_dia_lunes},
    {'type': 'eq', 'fun': restriccion_dia_martes},
    {'type': 'eq', 'fun': restriccion_dia_miercoles},
    {'type': 'eq', 'fun': restriccion_dia_jueves},
    {'type': 'eq', 'fun': restriccion_dia_viernes},
    {'type': 'eq', 'fun': restriccion_dia_sabado},
    {'type': 'eq', 'fun': restriccion_dia_domingo}
]

# Optimización para minimizar costos respetando las restricciones
resultado = minimize(costo_nomina, x0, bounds=bounds, constraints=restricciones)

# Mostrar resultados
resultado

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 8160.000000000003
       x: [ 0.000e+00  1.000e+00  2.000e+00  2.000e+00  1.000e+00
            1.332e-15  0.000e+00]
     nit: 2
     jac: [ 1.280e+03  1.360e+03  1.360e+03  1.360e+03  1.360e+03
            1.280e+03  1.200e+03]
    nfev: 16
    njev: 2

### **4. La solución obtenida, la cual debe ser indicada claramente en el informe.**

In [404]:
solucion_optima = np.round(resultado.x).astype(int)
costo_minimo = costo_nomina(solucion_optima)

planes = ["A (Domingo-Lunes)", "B (Lunes-Martes)", "C (Martes-Miércoles)", 
          "D (Miércoles-Jueves)", "E (Jueves-Viernes)", "F (Viernes-Sábado)", 
          "G (Sábado-Domingo)"]

print("=== Resultados de la Optimización ===")
for i in range(len(planes)):
    print(f"{planes[i]}: {solucion_optima[i]} enfermeras")

print(f"\nCosto total mínimo de la nómina: {costo_minimo} COP")


=== Resultados de la Optimización ===
A (Domingo-Lunes): 0 enfermeras
B (Lunes-Martes): 1 enfermeras
C (Martes-Miércoles): 2 enfermeras
D (Miércoles-Jueves): 2 enfermeras
E (Jueves-Viernes): 1 enfermeras
F (Viernes-Sábado): 0 enfermeras
G (Sábado-Domingo): 0 enfermeras

Costo total mínimo de la nómina: 8160 COP


### **5. Análisis de la solución obtenida desde el punto de vista matemático.**

In [400]:
def DerivativeMatrix(function, variables):
    """
    Calcula la matriz Jacobiana de derivadas parciales de un conjunto de funciones respecto a un conjunto de variables.
    """
    m = len(variables)
    n = len(function)
    matrix = sym.zeros(n, m)
    for fun in range(n):
        for variable in range(m):
            f = function[fun]
            v = variables[variable]
            matrix[fun, variable] = f.diff(v)
    return matrix

def GetLagrange(f, constraints):
    """
    Aplica el método de Lagrange para optimización con restricciones de igualdad.
    """
    # Definir variables de decisión
    x0, x1, x2, x3, x4, x5, x6 = sym.symbols('x0 x1 x2 x3 x4 x5 x6')
    lambda0, lambda1, lambda2, lambda3, lambda4, lambda5, lambda6 = sym.symbols('lambda0 lambda1 lambda2 lambda3 lambda4 lambda5 lambda6')
    
    # Construcción del sistema de ecuaciones de Lagrange
    g0, g1, g2, g3, g4, g5, g6 = constraints
    # Derivadas parciales de la función objetivo respecto a cada variable
    df_dx0 = f.diff(x0) - (lambda0 * g0.diff(x0) + lambda1 * g1.diff(x0) + lambda2 * g2.diff(x0) + lambda3 * g3.diff(x0) + lambda4 * g4.diff(x0) + lambda5 * g5.diff(x0) + lambda6 * g6.diff(x0))
    df_dx1 = f.diff(x1) - (lambda0 * g0.diff(x1) + lambda1 * g1.diff(x1) + lambda2 * g2.diff(x1) + lambda3 * g3.diff(x1) + lambda4 * g4.diff(x1) + lambda5 * g5.diff(x1) + lambda6 * g6.diff(x1))
    df_dx2 = f.diff(x2) - (lambda0 * g0.diff(x2) + lambda1 * g1.diff(x2) + lambda2 * g2.diff(x2) + lambda3 * g3.diff(x2) + lambda4 * g4.diff(x2) + lambda5 * g5.diff(x2) + lambda6 * g6.diff(x2))
    df_dx3 = f.diff(x3) - (lambda0 * g0.diff(x3) + lambda1 * g1.diff(x3) + lambda2 * g2.diff(x3) + lambda3 * g3.diff(x3) + lambda4 * g4.diff(x3) + lambda5 * g5.diff(x3) + lambda6 * g6.diff(x3))
    df_dx4 = f.diff(x4) - (lambda0 * g0.diff(x4) + lambda1 * g1.diff(x4) + lambda2 * g2.diff(x4) + lambda3 * g3.diff(x4) + lambda4 * g4.diff(x4) + lambda5 * g5.diff(x4) + lambda6 * g6.diff(x4))
    df_dx5 = f.diff(x5) - (lambda0 * g0.diff(x5) + lambda1 * g1.diff(x5) + lambda2 * g2.diff(x5) + lambda3 * g3.diff(x5) + lambda4 * g4.diff(x5) + lambda5 * g5.diff(x5) + lambda6 * g6.diff(x5))
    df_dx6 = f.diff(x6) - (lambda0 * g0.diff(x6) + lambda1 * g1.diff(x6) + lambda2 * g2.diff(x6) + lambda3 * g3.diff(x6) + lambda4 * g4.diff(x6) + lambda5 * g5.diff(x6) + lambda6 * g6.diff(x6))
    
    equations = [df_dx0, df_dx1, df_dx2, df_dx3, df_dx4, df_dx5, df_dx6, g0, g1, g2, g3, g4, g5, g6]
    
    values = sym.solve(equations, (x0, x1, x2, x3, x4, x5, x6, lambda0, lambda1, lambda2, lambda3, lambda4, lambda5, lambda6))
    return values


# Definir las variables de decisión
x0, x1, x2, x3, x4, x5, x6 = sym.symbols('x0 x1 x2 x3 x4 x5 x6')

# Definir la función objetivo
f = 1280*x0 + 1360*x1 + 1360*x2 + 1360*x3 + 1360*x4 + 1280*x5 + 1200*x6

# Definir las restricciones de igualdad
constraints = [
    x1 + x2 + x3 + x4 + x5 - 6,
    x2 + x3 + x4 + x5 + x6 - 5,
    x0 + x3 + x4 + x5 + x6 - 3,
    x0 + x1 + x4 + x5 + x6 - 2,
    x0 + x1 + x2 + x5 + x6 - 3,
    x0 + x1 + x2 + x3 + x6 - 5,
    x0 + x1 + x2 + x3 + x4 - 6
]

# Aplicar el método de Lagrange para encontrar los valores óptimos
values = GetLagrange(f, constraints)
values


{lambda0: 320,
 lambda1: 240,
 lambda2: 240,
 lambda3: 240,
 lambda4: 240,
 lambda5: 240,
 lambda6: 320,
 x0: 0,
 x1: 1,
 x2: 2,
 x3: 2,
 x4: 1,
 x5: 0,
 x6: 0}

Una forma analítica de abordar problemas de optimización con restricciones es utilizando los multiplicadores de Lagrange. Este método nos permite encontrar el mejor resultado posible respetando las condiciones del problema. En este caso, queremos minimizar el costo de la nómina de enfermeras asegurando que haya suficientes cada día.

Al resolver el sistema de ecuaciones, encontramos una única solución que garantiza el costo más bajo, lo cual tiene sentido porque el método de Lagrange busca justamente eso. La solución obtenida es:

x0: 0

x1: 1

x2: 2

x3: 2

x4: 1

x5: 0

x6: 0

Esta solución es la misma que habíamos obtenido antes con la implementación en spicy, lo que implicaria que es correcta.

### **6. Análisis de la solución desde el punto de vista del contexto del problema.**

**Interpretación del resultado:**

- **Turnos asignados:**  
  - Turno B (descanso Lunes-Martes): 1 enfermera  
  - Turno C (descanso Martes-Miércoles): 2 enfermeras  
  - Turno D (descanso Miércoles-Jueves): 2 enfermeras  
  - Turno E (descanso Jueves-Viernes): 1 enfermera  
  - Los turnos A, F y G tienen 0 enfermeras asignadas.

**Análisis de la cobertura diaria:**  
Recordando que cada turno significa que la enfermera trabaja los 5 días restantes, tenemos:  
- **Domingo:** Lo trabajan las enfermeras de los turnos B, C, D y E, lo que suma 1 + 2 + 2 + 1 = 6 enfermeras.  
- **Lunes:** Se trabaja en los turnos C, D y E (5 enfermeras en total).  
- **Martes:** Solo trabajan en D y E (2 + 1 = 3 enfermeras).  
- **Miércoles:** Trabajan en B y E (1 + 1 = 2 enfermeras).  
- **Jueves:** Se cuenta con B y C (1 + 2 = 3 enfermeras).  
- **Viernes:** Se suman B, C y D (1 + 2 + 2 = 5 enfermeras).  
- **Sábado:** Trabajan B, C, D y E (1 + 2 + 2 + 1 = 6 enfermeras).

Esto se ajusta a la demanda original:  
- Domingo: 5 enfermeras requeridas (se cubren 6)  
- Lunes: 5 enfermeras requeridas (se cubren 5)  
- Martes: 3 enfermeras requeridas (se cubren 3)  
- Miércoles: 2 enfermeras requeridas (se cubren 2)  
- Jueves: 3 enfermeras requeridas (se cubren 3)  
- Viernes: 5 enfermeras requeridas (se cubren 5)  
- Sábado: 6 enfermeras requeridas (se cubren 6)

**Desde el punto de vista del contexto del problema:**  

La asignación de turnos, vista en términos de días de descanso, garantiza que cada día se cuente con el número exacto de enfermeras necesarias para atender la demanda de la clínica (excepto el domingo). Además, cada enfermera, según el turno asignado, tiene dos días consecutivos libres, lo que cumple con la preferencia de trabajar cinco días seguidos y descansar dos días. Por otro lado, la solución logra optimizar el costo total de la nómina, resultando en 8160 COP, lo que demuestra que se ha cubierto la demanda con el menor gasto posible y cumpliendo las condiciones anteriores.
